# Comparaison de plusieurs répertoires

## I./ Coefficient de correlaton de Pearson 

In [2]:
import numpy as np
from repertoire_summary import *

def cor(list_rep) :
    """
    Calculate the Pearson correlation coefficient between several vectors of diversity
    obtained with hill_diversity() function
    input : list of repertoire as list of dataframe
    output : Pearson coefficient correlation matrix
    """
    
    list_div = []
    
    for i in list_rep :
        list_div.append(hill_diversity(i))
    
    #r = (np.sum((x_vect-np.mean(x_vect))*(y_vect-np.mean(y_vect)))) / np.sqrt(np.sum(((x_vect-np.mean(x_vect))**2))*np.sum(((y_vect-np.mean(y_vect))**2)))
    r = np.corrcoef(list_div)
    
    return r
    

### Import TSV files

In [2]:
import csv
import pandas as pd

# monoclonal
data_mono = pd.read_csv('../data/l0036_mono_airr.tsv', sep='\t')

# oligoclonal
data_oligo = pd.read_csv('../data/l0036_oligo_airr.tsv', sep='\t')

# polyclonal
data_poly = pd.read_csv('../data/l0036_poly_airr.tsv', sep='\t')

In [3]:
cor([data_mono, data_oligo, data_poly])

array([[1.        , 0.99880041, 0.99938683],
       [0.99880041, 1.        , 0.99936601],
       [0.99938683, 0.99936601, 1.        ]])

## II./ Jensen-Shannon Divergence

In [49]:
def JSD(x, y) :
    """
    Calculate Jensen-Shannon Divergence between two distance pairwise distribution 
    input1 x : first repertoire 
    input2 y : second repertoire
    output : Jensen-Shannon Divergence
    """
    
    pdd_x = PDDistribution(x)
    pdd_y = PDDistribution(y)
    
    x_vect = count_values(np.array(pdd_x))
    y_vect = count_values(np.array(pdd_y))
    
    if len(x_vect) > len(y_vect) :
        for i in range(len(x_vect)-len(y_vect)) :
            y_vect.append(0)
    elif len(y_vect) > len(x_vect) :
        for i in range(len(y_vect)-len(x_vect)) :
            x_vect.append(0)
    
    # add eps=10e-10 to avoid log(0) error
    x_vect = np.array(x_vect)+10e-10
    y_vect = np.array(y_vect)+10e-10
    # Normalize to obtain probability vector
    x_vect = x_vect / sum(x_vect)
    y_vect = y_vect / sum(y_vect)
    
    M = 0.5 * (x_vect + y_vect)
    D_xM = np.sum(x_vect * np.log(x_vect/M))
    D_yM = np.sum(y_vect * np.log(y_vect/M))
    
    JSD = 0.5 * D_xM + 0.5 * D_yM
    
    return JSD

In [70]:
JSD(data_mono, data_poly)

0.29002106800385485

In [54]:
def JSD_matrix(list_rep) :
    """
    Calculate the JSD value for each pair of repertoire
    input list_rep :list of repertoire dataframe
    output : matrix containing JSD values 
    """
    
    N = len(list_rep)
    
    jsd_mat = np.zeros((N, N))
    
    for i in range(N) :
        for j in range(N) :
            jsd_mat[i][j] = JSD(list_rep[i], list_rep[j])
            
    print(jsd_mat)
    

In [55]:
JSD_matrix([data_mono, data_oligo, data_poly])

[[0.         0.19241391 0.29002107]
 [0.19241391 0.         0.10134753]
 [0.29002107 0.10134753 0.        ]]


## III./ Repertoire Overlap

In [66]:
def overlap(x, y, option='sequence') :
    """
    Calculate the repertoire overlap value between two repertoires
    option argument defines which column is used, 'sequence' by default
    input1 x : first repertoire
    input2 y : second repertoire
    input3 option : name of which column is used
    output value : repertoire overlapv alue
    """
    # delete duplicate
    x_rep = np.array(x.drop_duplicates(subset=[option])[option])
    y_rep = np.array(y.drop_duplicates(subset=[option])[option])
    
    C = x_rep[np.in1d(x_rep, y_rep)]
    
    value = len(C) / min(len(x_rep), len(y_rep))
    
    return value
    

In [67]:
overlap(data_oligo, data_poly)

0.011976047904191617

In [68]:
def overlap_matrix(list_rep, option = 'sequence') :
    """
    Calculate the repertoire overlap value for eawh pair of repertoire in list_rep
    input1 list_rep : list of repertoire dataframe
    input2 option : name of which colmun is used, 'sequence' by default
    output overlap_mat : matrix containing overlap values 
    """
    N = len(list_rep)
    
    overlap_mat = np.zeros((N,N))
    
    for i in range(N) :
        for j in range(N) :
            overlap_mat[i][j] = overlap(list_rep[i], list_rep[j])
            
    return overlap_mat

In [69]:
overlap_matrix([data_mono, data_oligo, data_poly])

array([[1.        , 0.        , 0.        ],
       [0.        , 1.        , 0.01197605],
       [0.        , 0.01197605, 1.        ]])